# MLPs with PyTorch
This is my practice of the original by S. Raschka found [here](https://github.com/rasbt/deep-learning-book/blob/master/code/model_zoo/pytorch_ipynb/multilayer-perceptron.ipynb) except for the use of Fashion-MNIST.

In [1]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import numpy as np

%matplotlib inline

In [2]:
############# Parameters #############

# Hyperparameters
learning_rate = 0.01
num_epochs = 10
batch_size = 64

# Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10

############# Fashion MNIST #############

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.FashionMNIST(root='data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

test_dataset = datasets.FashionMNIST(root='data',
                                     train=False,
                                     transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=False)


# Check datasets
for images, labels in train_loader:
    print('image batch shape = ', images.shape),
    print('image label shape = ', labels.shape)
    break

image batch shape =  torch.Size([64, 1, 28, 28])
image label shape =  torch.Size([64])


In [3]:
############# Model #############

class MultilayerPerceptron(torch.nn.Module):
    """This architecture is a ensely connected 2 hidden layer network."""
    
    def __init__(self, num_features, num_classes):
        """Constructor"""
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        # The following two lines are not necessary
        self.linear_1.weight.data.normal_(0.0, 0.1)
        self.linear_1.bias.data.normal_(0.0, 0.01)
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.data.normal_(0.0,0.1)
        self.linear_2.bias.data.normal_(0.0, 0.01)
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.data.normal_(0.0, 0.1)
        self.linear_out.bias.data.normal_(0.0, 0.01)
        
    def forward(self, x):
        # TODO try a sequential setup http://pytorch.org/docs/master/nn.html#torch.nn.Module.modules
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

### Create model
torch.manual_seed(42)
model = MultilayerPerceptron(num_features=num_features,
                            num_classes=num_classes)


if torch.cuda.is_available():
    model.cuda()

############# Define cost function and optimizer #############
cost_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [4]:
from torch.autograd import Variable
import torch.nn.functional as F

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = Variable(features.view(-1, 28*28))
        if torch.cuda.is_available():
            features = features.cuda()
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas.data, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels.cpu() == targets).sum()
    return correct_pred / num_examples * 100

for epoch in range(num_epochs):
    """Train"""
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = Variable(features.view(-1, 28*28))
        targets = Variable(targets)
        
        if torch.cuda.is_available():
            features, targets = features.cuda(), targets.cuda()
            
        ### Forward
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        # Sets gradients of all model parameters to zero:
        optimizer.zero_grad()
        
        # Backprop
        cost.backward()
        
        ### Update model parameters
        optimizer.step()
        
        ### Logging
        if not batch_idx % 100:
            print('Epoch:  %03d/%03d | Batch:  %03d/%03d | Cost:  %.4f' %
                 (epoch+1, num_epochs, batch_idx,
                 len(train_dataset)//batch_size, cost.data[0]))
            
    print('Epoch:  %03d/%03d training accuracy %.2f' %
         (epoch+1, num_epochs,
         compute_accuracy(model, train_loader)))

Epoch:  001/010 | Batch:  000/937 | Cost:  2.7237
Epoch:  001/010 | Batch:  100/937 | Cost:  1.0741
Epoch:  001/010 | Batch:  200/937 | Cost:  0.8383
Epoch:  001/010 | Batch:  300/937 | Cost:  0.6048
Epoch:  001/010 | Batch:  400/937 | Cost:  0.6502
Epoch:  001/010 | Batch:  500/937 | Cost:  0.9140
Epoch:  001/010 | Batch:  600/937 | Cost:  0.6373
Epoch:  001/010 | Batch:  700/937 | Cost:  0.5314
Epoch:  001/010 | Batch:  800/937 | Cost:  0.6751
Epoch:  001/010 | Batch:  900/937 | Cost:  0.6222
Epoch:  001/010 training accuracy 79.87
Epoch:  002/010 | Batch:  000/937 | Cost:  0.7609
Epoch:  002/010 | Batch:  100/937 | Cost:  0.5350
Epoch:  002/010 | Batch:  200/937 | Cost:  0.5014
Epoch:  002/010 | Batch:  300/937 | Cost:  0.8174
Epoch:  002/010 | Batch:  400/937 | Cost:  0.5565
Epoch:  002/010 | Batch:  500/937 | Cost:  0.6584
Epoch:  002/010 | Batch:  600/937 | Cost:  0.4616
Epoch:  002/010 | Batch:  700/937 | Cost:  0.6505
Epoch:  002/010 | Batch:  800/937 | Cost:  0.5436
Epoch:  00

In [5]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 84.79%
